<h3><b> Video Compression Using GPU</h3>



In [1]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
#@markdown <h4><b>←</b> ឵឵Click Here to <b>Force Remount</b> your Google Drive</h4>
#@markdown <h5><i>(Only use if needed)</i></h5>
drive.mount("/content/drive", force_remount=True)

In [ ]:
#@markdown <h4> ឵឵ Pre-Requisites</h4>
!nvidia-smi
!git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git
!cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/
!ffmpeg -y -encoders=nvenc_hevc

In [2]:
!sudo apt install mediainfo

In [ ]:
#@markdown <h3>⬅️ Click Here to Check GPU</h3>
!nvidia-smi -L

###» Display Media File Codecs + Metadata

In [ ]:
import os, sys, re

inputFile = "/content/drive/MyDrive/Input folder/dolby atmos.mkv" #@param {type:"string"}

!mediainfo "$inputFile" 

### <font face="Rubik" size="4">» </font><font face="Rubik" size="5">Video Encode Block</font>


In [ ]:
#@markdown <h3><b>Full Folder Encode :</h3>

import os, sys, re

input_folder= '/content/drive/MyDrive/Input folder' #@param {type:"string"}
output_folder= '/content/drive/MyDrive/output folder' #@param {type:"string"}

#@markdown <h3><b>Choose your encode setting :</h5>
#@markdown <h><i>main = default | main10 = HEVC 10 Bit</i>
encode_setting = "AAC 94k (Video Copy)" #@param ["HEVC AAC 94k", "HEVC AAC_adtstoasc", "HEVC (Audio Copy)", "AAC 94k (Video Copy)"]
set_profile = "main" #@param ["main", "main10"]
set_bitrate= '1M' #@param {type:"string"}
output_file_type = 'mp4' #@param ["mkv", "mp4"]
#@markdown <h3><b>Video quality :</h3>
#@markdown <h><i>0 (Best) to 51 (Worst)</i><h>
set_cq = '22'#@param {type:"string"}
set_qmin = '0'#@param {type:"string"}
set_qmax = '22'#@param {type:"string"}
my_suffixes = (".mp4", ".mov", ".mkv", ".avi", ".ts", ".flv", ".webm", ".wmv", ".mpg", ".m4v", ".f4v", ".m2ts", ".mpeg", ".3gp", ".MP4", ".MOV", ".MKV", ".AVI", ".TS", ".FLV", ".WEBM", ".WMV", ".MPG", ".M4V", ".F4V", ".M2TS", ".MPEG", ".3GP")
from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

if encode_setting == 'HEVC AAC 94k':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuda -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v $set_bitrate -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -c:a aac -b:a 94k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'HEVC AAC_adtstoasc':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuda -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v $set_bitrate -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -bsf:a aac_adtstoasc "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"

if encode_setting == 'HEVC (Audio Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error -stats -hwaccel cuda -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v hevc_nvenc -preset:v slow -rc vbr -cq $set_cq -qmin $set_qmin -qmax $set_qmax -b:v 0  -profile:v $set_profile -init_qpP 23 -init_qpB 25 -init_qpI 21 -multipass 2 -b_ref_mode 2 -c:a copy "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
                
if encode_setting == 'AAC 94k (Video Copy)':
 for filename in os.listdir(input_folder):
    if (filename.endswith(my_suffixes)):
        cmd = !ffmpeg -v error  -stats -hwaccel cuvid -i "$input_folder/{filename}" -metadata comment="HEVC-CQ$set_cq-AAC" -c:v copy -c:a aac -b:a 94k "$output_folder/{filename.rpartition('.')[0]}.$output_file_type"
                     

In [ ]:
#@markdown <h3><b>output mediainfo</h3>
import os, sys, re

inputFile = "/content/drive/MyDrive/output folder/dolby atmos.mp4" #@param {type:"string"}

!mediainfo "$inputFile" 

### <font face="Rubik" size="4">» </font><font face="Rubik" size="5">Extract + Upload Frames from Video to ImgBB</font>
<font size="3"><i>Create your own ImgBB API Key here: https://api.imgbb.com/</font>

In [ ]:
ImgBB_API_Key="00a8f49cf0453ac7fd3016e73b72155c" #@param {type:"string"}
inputFile = "/content/drive/MyDrive/output folder/dolby atmos.mp4" #@param {type:"string"}
!mkdir -p "/content/frames/"

import os
import base64
import requests
from IPython.display import clear_output


!rm -f /content/frames/*

def upload_to_ImgBB(img):
  with open(img, "rb") as file:
    url = "https://api.imgbb.com/1/upload"
    payload = {
        "key": ImgBB_API_Key,
        "image": base64.b64encode(file.read()),
    }
    res = requests.post(url, payload).json()
    return res['data']['url']

clear_output()
!ffmpeg -hide_banner -ss 00:20.0 -i "$inputFile" -vframes 1 -q:v 2 -y "frame1.png"
clear_output()
!ffmpeg -hide_banner -ss 00:40.0 -i "$inputFile" -vframes 1 -q:v 2 -y "frame2.png"
clear_output()
!ffmpeg -hide_banner -ss 00:50.0 -i "$inputFile" -vframes 1 -q:v 2 -y "frame3.png"
clear_output()

%cd "/content/frames/"
print("Uploading Images...")
Screenshots=[]
for i in range(1,4):
  Screenshots.append(upload_to_ImgBB("frame{}.png".format(i)))
%cd "/"
clear_output()

for i in range(3):
 print(format(Screenshots[i]))

https://i.ibb.co/JqRBCrm/b3cff3f519fd.png
https://i.ibb.co/RhSb7x9/b4bbc549d779.png
https://i.ibb.co/D40p9RK/0d0eadcba8ec.png


In [ ]:

ImgBB_API_Key="00a8f49cf0453ac7fd3016e73b72155c" #@param {type:"string"}
inputFile = "3.mkv" #@param {type:"string"}
!mkdir -p "/frames/"

import os
import base64
import requests
from IPython.display import clear_output


!rm -f /frames/*

def upload_to_ImgBB(img):
  with open(img, "rb") as file:
    url = "https://api.imgbb.com/1/upload"
    payload = {
        "key": ImgBB_API_Key,
        "image": base64.b64encode(file.read()),
    }
    res = requests.post(url, payload).json()
    return res['data']['url']

clear_output()
!ffmpeg -hide_banner -ss 00:20.0 -i "$inputFile" -vframes 1 -q:v 2 -y "\frames\frame1.png"
clear_output()
!ffmpeg -hide_banner -ss 00:40.0 -i "$inputFile" -vframes 1 -q:v 2 -y "\frames\frame2.png"
clear_output()
!ffmpeg -hide_banner -ss 00:50.0 -i "$inputFile" -vframes 1 -q:v 2 -y "\frames\frame3.png"
clear_output()

%cd "\frames\"
print("Uploading Images...")
Screenshots=[]
for i in range(1,4):
  Screenshots.append(upload_to_ImgBB("frame{}.png".format(i)))
%cd "\"
clear_output()

for i in range(3):
 print(format(Screenshots[i]))

In [ ]:
%cd C:\ESD
